# Calibrate Pi Pulses
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial introduces how to calibrate a $\pi$ pulse by varying the amplitude of the drive pulse. The outline of this tutorial is as follows:
- Preparation
- Introduction
- Define the system's Hamiltonian
- Sweep amplitudes
- Cosine regression
- Summary

## Preparation
After you have successfully installed Quanlse on your device, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import the Hamiltonian module
from Quanlse.Utils import Hamiltonian as qham 

# Import related packages
from Quanlse.Utils.Operator import duff, driveX
from Quanlse.Utils.Waveforms import gaussian

# Import simulator interface for Quanlse Cloud Service
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# Import numpy
from numpy import linspace, pi, dot, array, cos

# Import matplotlib
import matplotlib.pyplot as plt

# Import curve_fit function from scipy
from scipy.optimize import curve_fit



## Introduction

Calibrating $\pi$ pulses is one of the most fundamental operations in quantum computing, because one of the most fundamental gates, the X gate, requires a $\pi$ pulse input onto the X channel. Further, it also serves an important role in calibrating actual hardware. Thus, this tutorial will demonstrate how to calibrate a $\pi$ pulse using Quanlse.

## Define the system Hamiltonian

In the field of quantum control, it is a common practice to describe a quantum system with its Hamiltonian. Generally, a system Hamiltonian consists of two terms, the time-independent and the time-dependent terms:

$$
\hat{H}_{\rm total}(t) = \hat{H}_{\rm drift} + \hat{H}_{\rm ctrl }(t) .
$$


The users could easily define the Hamiltonian for a multi-qubit system using the `Hamiltonian` module in Quanlse. First, we will use the `Hamiltonian` module to initialize a system Hamiltonian. We start with a single-qubit system with three energy levels. The system Hamiltonian can be written as:

$$
\hat{H} = \alpha_q \hat{a}^{\dagger}\hat{a}^{\dagger}\hat{a}\hat{a} + \frac{1}{2} c(t) \cos(\phi) (\hat{a}+\hat{a}^{\dagger}).
$$

Here, the $\alpha_q$ is the anharmonicity between the two lowest transisition energies. $c(t)$ indicates the pulse envelope function; and $\phi$ is the pulse phase. $\hat{a}^{\dagger}=|1\rangle\langle 0|+\sqrt{2}|2\rangle\langle 1|$ and $\hat{a}=|0\rangle\langle 1|+\sqrt{2}|1\rangle\langle 2|$ are respectively the creation and annihilation operators.

Here, we will demonstrate how to define such a Hamiltonian using Quanlse. We will first initialize the Hamiltonian dictionary using the following code:

In [ ]:
ham = qham.createHam(title="example", dt=0.2, qubitNum=1, sysLevel=3)

The above `createHam()` function returns an empty Hamiltonian dictionary. Its parameters include a user-defined title, sampling period, qubit number, and the system's energy levels to consider.

Then we could add terms to the empty Hamiltonian dictionary using the two functions below. The function `addDrift()` adds drift operators to the Hamiltonian while the `addControl()` function adds the operators associated with the control pulses. Both functions require a `Hamiltonian` dictionary, a user-defined name, the qubit(s) index(es) which the term is acting upon, the according operators (we have conveniently provided the `Operator` module which includes many commonly-used operators), and the amplitude (only for the drift term):

In [ ]:
alphaq = - 0.22 * (2 * pi)  # unit is GHz
qham.addDrift(ham, "drift", onQubits=0, matrices=duff(3), amp=alphaq)
qham.addControl(ham, "ctrl", onQubits=0, matrices=driveX(3))

Then, the user could use the `printHam()` function to display the properties of the Hamiltonian. The `printHam()` function takes a Hamiltonian dictionary:

In [ ]:
qham.printHam(ham)

Here we could conveniently use `Operator`'s method `duff(n)` to define the $n$-dimensional $\hat{a}^{\dagger}\hat{a}^{\dagger}\hat{a}\hat{a}$, and `driveX(n)` to define the $n$-dimensional $\frac{1}{2}(\hat{a}+\hat{a}^{\dagger})$. After appending the control term to the Hamiltonian, we need to add the effective pulse:

$$
c(t) = A e^{-(t-\tau)^2/2\sigma^2}.
$$

We achieve this by using the `setWave()` function. The `setWave()` function takes a Hamiltonian dictionary, the name of the term in the Hamiltonian, waveform (Quanlse supports multiple waveforms' definitions), the parameters needed to define the wave, and lastly, the initial time and the duration of the wave.

In [ ]:
qham.setWave(ham, "ctrl", f="gaussian", para={"a": 1, "tau":10, "sigma":3}, t0=0, t=20)

Here, we have just defined a complete quantum system and the parameters regarding controlling the system. We can visualize the pulse using the provided `plotWaves()` function. The `plotWaves()` function plots the pulses by taking a Hamiltonian dictionary and the according terms' names. The function also includes an optional bool parameter `dark`, which enables a dark-themed mode. Moreover, the user can use the `color` parameter to specify colors for individual pulses (the colors will repeat if there are more pulses than colors).

In [ ]:
qham.plotWaves(ham, "ctrl", dark=True, color=['mint'])

Then we can use the `simulate()` function to simulate the evolution, and obtain the unitary matrix of the system evolution.

In [ ]:
result = qham.simulate(ham, recordEvolution=False)
result

## Sweep amplitudes

With fixed pulse duration $t_g$, we can sweep the pulse's amplitudes $a$, and find the amplitude $a_{\pi}$ of the according $\pi$ pulse.

We first create a list of 200 points between -1 and 1, representing the pulse's amplitudes. 

In [ ]:
# Initilize the pulse's amplitudes
alist = linspace(-1.0, 1.0, 200)
pop0_list = []
pop1_list = []
pop2_list = []

Then, we can obtain the according population for each state by simulating the evolution of the Hamiltonian defined in the previous section. The calculation usually takes a long time to process on local devices; however, we provide a cloud computing service that could speed up this process significantly. To use Quanlse Cloud Service, the users can get a token from http://quantum-hub.baidu.com and submit the job onto Quanlse's server. Note that Quanlse supports the submission of batches of job, which could further optimize the allocation of resources.

In [ ]:
# Calibrate a Pi Pulse
jobList = []
for a in alist:
    # Configure pulse parameters
    jobTemp = []
    jobTemp.append(qham.makeWaveData(ham, "ctrl", f=gaussian, para={"a": a, "tau": 10, "sigma": 3}, t0=0, t=20))
    # Run similator
    jobList.append(jobTemp)

# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ''

# Submit batch jobs to Quanlse Cloud Service
resultList = runHamiltonian(ham, jobList=jobList)

# Calculate populations
for result in resultList:
    final_state = dot(result["unitary"], array([1, 0, 0], dtype=complex))
    pop0_list.append(abs(final_state[0])**2)
    pop1_list.append(abs(final_state[1])**2)
    pop2_list.append(abs(final_state[2])**2)

# Plot graph
plt.plot(alist, pop0_list, label="Ground state")
plt.plot(alist, pop1_list, label="1st excited state")
plt.plot(alist, pop2_list, label="2nd excited state")
plt.xlabel("Amplitude")
plt.ylabel("Population of different states")
plt.legend()
plt.show()

## Cosine regression

Now, we have a series of discrete points; however, we need to fit those points with a cosine function in order to find the amplitude of the $\pi$ pulse. To fit the resulting $|0\rangle$ population, we use the `optimize.curve_fit()` method in `Scipy`. We first define the following function:

In [ ]:
def fit_function(x_values, y_values, init_params):
    def fit_func(x, A, B, period, phi):
        return A * cos(2 * pi * x / period - phi) + B
    fitparams, _ = curve_fit(fit_func, x_values, y_values, init_params, bounds=(0, [2.0, 2.0, 2.0, 2.0]))
    y_fit = fit_func(x_values, *fitparams)
    return fitparams, y_fit

Then we run the regression function to obtain the result:

In [ ]:
fit_params, y_fit = fit_function(alist, pop0_list, [0.5, 0.5, 0.8, 0])

# Plot graph
plt.scatter(alist, pop0_list, label="Samples")
plt.plot(alist, y_fit, color="red", label="Fit curve")
plt.xlabel("Amplitude")
plt.ylabel("Population of grounf state")
plt.legend()
plt.show()
print(f"Period is {fit_params[2]}")
print(f"Pi pulse amplitude is {fit_params[2] / 2}")

By the cosine regression, we have identified the corresponding amplitude of the $\pi$ pulse.

## Summary
After reading this tutorial on calibrating $\pi$ pulses, the users could follow this link [tutorial-calibrate-pi-pulses.ipynb](https://github.com/baidu/Quanlse/tree/master/Tutorial/EN/tutorial-pi-pulse.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result.